In [ ]:
import heapq
import time
import random
import matplotlib.pyplot as plt
import math

class HoffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_hoffman_tree(symbols):
    heap = [HoffmanNode(char, freq) for char, freq in symbols.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        internal = HoffmanNode(None, left.freq + right.freq)
        internal.left = left
        internal.right = right

        heapq.heappush(heap, internal)

    return heap[0]

def generate_codes(root, current_code, codes):
    if root is None:
        return

    if root.char is not None:
        codes[root.char] = current_code
        return

    generate_codes(root.left, current_code + "0", codes)
    generate_codes(root.right, current_code + "1", codes)

def hoffman_coding(symbols):
    root = build_hoffman_tree(symbols)
    codes = {}
    generate_codes(root, "", codes)
    return codes

def measure_runtime_with_high_precision_and_overhead(n, trials=5):
    total_time = 0
    for _ in range(trials):
        symbols = {chr(65 + i % 26): random.randint(1, 1000) for i in range(n)}
        start_time = time.perf_counter()
        hoffman_coding(symbols)
        end_time = time.perf_counter()
        total_time += (end_time - start_time) * 1e9
    return total_time / trials

def estimate_constant_overhead(trials=10):
    total_time = 0
    for _ in range(trials):
        symbols = {chr(65 + i % 26): 1 for i in range(26)}
        start_time = time.perf_counter()
        hoffman_coding(symbols)
        end_time = time.perf_counter()
        total_time += (end_time - start_time) * 1e9
    return total_time / trials

def main_with_improvements():
    input_sizes = [10000, 50000, 100000, 500000, 1000000, 2000000, 5000000, 10000000]
    runtimes = []
    theoretical_times = []
    trials = 10

    constant_overhead = estimate_constant_overhead(trials=trials)

    for size in input_sizes:
        runtime = measure_runtime_with_high_precision_and_overhead(size, trials=trials) - constant_overhead
        runtimes.append(runtime)

        theoretical_time = size * math.log2(size)
        theoretical_times.append(theoretical_time)

    avg_experimental = sum(runtimes) / len(runtimes)
    avg_theoretical = sum(theoretical_times) / len(theoretical_times)

    scaling_constant = avg_experimental / avg_theoretical

    scaled_theoretical_times = [scaling_constant * t for t in theoretical_times]

    plt.plot(input_sizes, runtimes, label='Experimental Values', marker='o')
    plt.plot(input_sizes, scaled_theoretical_times, label='Scaled Theoretical Values (n log n)', marker='x')
    plt.xlabel('Input Size')
    plt.ylabel('Time (ns)')
    plt.title('Comparison of Experimental and Scaled Theoretical Time Complexity')
    plt.legend()
    plt.grid(True)
    plt.show()

main_with_improvements()
